<a href="https://colab.research.google.com/github/carlosprr29/ai-progetto-spagnoli/blob/main/notebooks/04_Testing_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. INSTALLATION AND BOOKSTORES
!pip install -q transformers datasets torch scikit-learn
import pandas as pd
import numpy as np
import random
import torch
import re
import shutil
from google.colab import drive
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
# 2. CONNECTION TO DRIVE AND CHARGING OF ALL MODELS
# -----------------------------------------------------------------
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("⏳ Loading all trained models... (this may take a while)")

# WELFake load
m_welfake = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Project_IA/model_WELFake_full').to(device)
t_welfake = BertTokenizer.from_pretrained('/content/drive/MyDrive/Project_IA/model_WELFake_full')
m_welfake_tit = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Project_IA/model_WELFake_titles').to(device)
t_welfake_tit = BertTokenizer.from_pretrained('/content/drive/MyDrive/Project_IA/model_WELFake_titles')

# ISOT load
m_isot = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Project_IA/model_isot_full').to(device)
t_isot = BertTokenizer.from_pretrained('/content/drive/MyDrive/Project_IA/model_isot_full')
m_isot_tit = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Project_IA/model_isot_titles').to(device)
t_isot_tit = BertTokenizer.from_pretrained('/content/drive/MyDrive/Project_IA/model_isot_titles')

# FUSION load
m_fusion = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Project_IA/model_fusion_full').to(device)
t_fusion = BertTokenizer.from_pretrained('/content/drive/MyDrive/Project_IA/model_fusion_full')
m_fusion_tit = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Project_IA/model_fusion_titles').to(device)
t_fusion_tit = BertTokenizer.from_pretrained('/content/drive/MyDrive/Project_IA/model_fusion_titles')

In [ ]:
# -----------------------------------------------------------------
# 2. INTERACTIVE CONTROL PANEL (@title)
# -----------------------------------------------------------------
# @title 🔍 Control Panel: Fake News Detector Pro

test_title = "Vladimir Putin, trapped by his own logic of war" # @param {type:"string"}
test_text = "Four years after launching his war against Ukraine, Vladimir Putin still has not secured the victory he had hoped for. When he sent his troops to attack Russia's neighbor on February 24, 2022, the Russian leader claimed his aim was to denazify and demilitarize a country whose very existence he denies. Seven months earlier, he had laid out his ambitions in an essay titled, On the Historical Unity of Russians and Ukrainians.This essay, published on the Kremlin's official website, clearly announced his special military operation, supposed to bring a change of power in Kyiv and demonstrate the greatness of eternal Russia in the face of what he viewed as an aggressive and decadent West. That objective remains far from achieved: Volodymyr Zelensky, the Ukrainian president, is still in office, and European support for Ukraine has held firm." # @param {type:"string"}
threshold = 0.65 # @param {type:"slider", min:0.5, max:0.95, step:0.05}
model_to_use = "Fusion" # @param ["WELFake", "ISOT", "Fusion"]

def detect_pro(title, text, sensitivity):
    # Dynamic selection of model pairs (Titles and Full) according to the parameter
    if model_to_use == "WELFake":
        m_f, t_f = m_welfake, t_welfake
        m_t, t_t = m_welfake_tit, t_welfake_tit
    elif model_to_use == "ISOT":
        m_f, t_f = m_isot, t_isot
        m_t, t_t = m_isot_tit, t_isot_tit
    elif model_to_use == "Fusion":
        m_f, t_f = m_fusion, t_fusion
        m_t, t_t = m_fusion_tit, t_fusion_tit

    classes = {0: "✅ REAL", 1: "🚨 FAKE"}

    # A. Model Prediction Headlines
    inputs_tit = t_t(title, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    m_t.eval()
    with torch.no_grad():
        out_tit = m_t(**inputs_tit)
        prob_tit = torch.nn.functional.softmax(out_tit.logits, dim=1)
        pred_tit = torch.argmax(prob_tit, dim=1).item()
        conf_tit = prob_tit[0][pred_tit].item()

    # B. Full Text Model Prediction
    input_combined = title + " " + text
    inputs_full = t_f(input_combined, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    m_f.eval()
    with torch.no_grad():
        out_full = m_f(**inputs_full)
        prob_full = torch.nn.functional.softmax(out_full.logits, dim=1)
        # Here we use the probability of FAKE (class 1) for the threshold logic.
        prob_fake_val = prob_full[0][1].item()
        pred_full = torch.argmax(prob_full, dim=1).item()

    # C. Threshold logic for final verdict (based on Full Text)
    if prob_fake_val >= sensitivity:
        result = "🚨 FAKE NEWS"
        colour = "\033[91m"
    elif prob_fake_val <= (1 - sensitivity):
        result = "✅ REAL"
        colour = "\033[92m"
    else:
        result = "🤔DOUBTFUL / INCONCLUSIVE"
        colour = "\033[93m"

    # D. Report
    print("\n" + "═"*60)
    print(f"📊 COMPARATIVE VERIFICATION REPORT ({model_to_use.upper()})")
    print("═"*60)
    print(f"📰 HEADLINE: {title[:85]}...")
    print(f"🤖 TITLES ONLY:   {classes[pred_tit]} ({conf_tit*100:.2f}% trust)")
    print(f"🤖 TEXTO COMPLETO: {colour}{result}\033[0m (Score Fake: {prob_fake_val*100:.2f}%)")
    print(f"⚙️  Current threshold:  {sensitivity}")
    print("═"*60)
    if pred_tit != pred_full and result != "🤔 DOUBTFUL / INCONCLUSIVE":
        print("📢 NOTE: The full text has corrected the prediction in the title.")

# Run the comparative detector with the news item from the panel
detect_pro(test_title, test_text, threshold)

In [4]:
def test_battery(title, text):
    """
    ALWAYS run the prediction on all three systems
    and display the compared results.
    """
    global model_to_use # We use the exact name of your variable.

    # We keep the original model to restore it later.
    original_selection = model_to_use

    # List with the exact names that your 'if' expects within detect_pro
    models = ["WELFake", "ISOT", "Fusion"]

    print("\n" + "═"*65)
    print(f"🔎 TRIPLE COMPARATIVE ANALYSIS")
    print(f"📰 HEADLINE: {title[:90]}...")
    print("═"*65)

    # Loop that runs through each model
    for m in models:
        model_to_use = m
        # We call your detect_pro function using 'threshold' (which is what it is called in your panel).
        detect_pro(title, text, threshold)

    # We restored the original model.
    model_to_use = original_selection
    print("═"*65 + "\n")

In [ ]:
#REAL NEWS FROM LE MONDE (FRENCH NEWSPAPER). URL:https://www.lemonde.fr/en/opinion/article/2026/02/23/vladimir-putin-trapped-by-his-own-logic-of-war_6750782_23.html
tit_1 = "Vladimir Putin, trapped by his own logic of war"
txt_1 = "Four years after launching his war against Ukraine, Vladimir Putin still has not secured the victory he had hoped for. When he sent his troops to attack Russia's neighbor on February 24, 2022, the Russian leader claimed his aim was to denazify and demilitarize a country whose very existence he denies. Seven months earlier, he had laid out his ambitions in an essay titled, On the Historical Unity of Russians and Ukrainians.This essay, published on the Kremlin's official website, clearly announced his special military operation, supposed to bring a change of power in Kyiv and demonstrate the greatness of eternal Russia in the face of what he viewed as an aggressive and decadent West. That objective remains far from achieved: Volodymyr Zelensky, the Ukrainian president, is still in office, and European support for Ukraine has held firm."

test_battery(tit_1, txt_1)

In [ ]:
#FAKE NEWS FROM theonion.com (The world's leading source of humorous fake news in English) URL:https://theonion.com/u-s-tourists-advised-to-temporarily-avoid-shootouts-with-mexican-drug-cartels/
tit_2 = "NASA Delays Space Walk After It Starts Snowing In Outer Space"
txt_2 = "With no choice but to suspend the planned outing and await more favorable conditions, NASA announced Wednesday it had delayed a space walk after a heavy snow began to fall in outer space. “Unfortunately, a front has moved into our solar system, causing a severe blizzard that forced us to postpone this morning’s scheduled space walk,” said NASA administrator Bill Nelson, who observed that four to five feet of snow had accumulated across much of the galaxy and that it was “quite blustery out there,” making it too difficult for astronauts to replace a malfunctioning antenna on the International Space Station. “Once the weather clears up, we’ll send our crew on a space walk with shovels and a bag of rock salt, and ensure the astronauts scrape off the windshield of their return spacecraft. In the meantime, we’ve ordered them to cozy up and build a fire inside the ISS.” At press time, NASA officials confirmed the treacherous condition had caused a capsule full of space tourists to skid out of Earth’s orbit and crash into the sun."

test_battery(tit_2, txt_2)

#NDON'T GET THE SATIRICAL TONE, THE HEADLINE LITERALLY SAYS: "La NASA retrasa una caminata espacial tras empezar a nevar en el espacio exterior"

In [ ]:
#REAL NEWS THE NEW YORK TIMES. URL: https://www.nytimes.com/2026/02/23/world/americas/el-mencho-mexico-cartel-military-raid.html
tit_3 = "Mexican Forces Say They Tracked El Mencho to Cabin by Following His Lover"
txt_3 = "To find Mexico’s most-wanted kingpin, security officials said, they did not follow the money or the trail of drugs. Instead, they said they followed his lover.It was she, the authorities said, who led them into the wooded mountains of Jalisco state, to the cabin where Nemesio Oseguera Cervantes — the notorious cartel leader known as El Mencho — was hiding since at least Friday.Quickly, a high-stakes plan was in motion to capture the head of the Jalisco New Generation Cartel, a man long considered one of the country’s most ruthless criminal figures, Mexican officials said Monday during a news conference.The operation culminated in a vicious firefight that killed several suspected cartel members, and by Sunday morning, Mr. Oseguera was dead after being fatally wounded in a shootout with Mexican authorities, the government said."

test_battery(tit_3, txt_3)

#ALL SYSTEMS ARE CORRECT

In [ ]:
#REAL NEWS THE NEW YORK TIMES. URL: https://www.nytimes.com/2026/02/24/world/europe/russia-ukraine-war-future.html
tit_4 = "How Russia Put Its Future at Risk by Remaking Its Economy for War"
txt_4 = "For four years, President Vladimir V. Putin of Russia has made the war against Ukraine the lodestar of his every move. The single-minded approach has helped Mr. Putin salvage what began as a disastrous invasion, get his troops back on the front foot and dictate demands in peace talks mediated by Washington. But his stubborn pursuit of the war has come at a huge cost. It has killed or wounded as many as 1.2 million Russians, by some estimates, while reordering Russia’s economy and society in ways that many economists believe jeopardize the nation’s future.“You have lots of money spent on tanks, shells, bombs, military benefits and other things — no long-lasting value, nothing that works on what we call development,” said Alexandra Prokopenko, a former Russian central bank official who is now a fellow at the Carnegie Russia Eurasia Center in Berlin."

test_battery(tit_4, txt_4)

In [ ]:
#FAKE NEWS FROM SNOPES.COM. URL: https://www.snopes.com/fact-check/swift-kelce-donate-baby-cancer/
tit_5 = "TAYLOR SWIFT AND TRAVIS KELCE SPENT 300,000 USD TO FIGHT FOR THE LIFE OF A 2-YEAR-OLD BABY WITH BRAIN CANCER — AND ANNOUNCED THE PLAN TO BUILD THE LARGEST FREE ORPHANAGE IN THE UNITED STATES WORTH 80 MILLION USD."
txt_5 = "Not a stage, not an award show, not the dazzling lights of Hollywood — what silenced America today was the compassion of two people standing at the center of the world. Within 24 hours, Taylor Swift and Travis Kelce quietly covered the entire 300,000 USD medical cost to save a 2-year-old girl battling brain cancer. That silent yet powerful act made social media explode, as millions witnessed a rare moment of kindness in a chaotic era. But the biggest shock was still to come. Immediately after the information was confirmed, the couple announced the plan to build the largest free orphanage in the United States, an 80-million-USD project for children without shelter and those in vulnerable situations. Not an empty promise — but a commitment, a concrete step toward real change."

test_battery(tit_5, txt_5)

In [ ]:
#FAKE NEWS FROM THEONION.COM URL: https://theonion.com/potomac-river-flooded-with-240-million-gallons-of-sewage/
tit_6 = "Potomac River Flooded With 240 Million Gallons Of Sewage"
txt_6 = "The Potomac River, a waterway that winds through the nation’s capital, is in the midst of an ecological crisis after one of the largest sewage spills in U.S. history, with over 240 million gallons of raw human waste threatening the health of the river and the safety of those who depend on it. What do you think?"

test_battery(tit_6, txt_6)

✅ The Good (Strengths)
Effectiveness in Real News: All three models show excellent accuracy in news from serious media outlets (NYT, Le Monde). They perfectly detect informative and neutral styles.

Full Text Correction: The "Full Text" model has been shown to often correct the "Headlines" model. This shows that AI does not just focus on clickbait but also analyses the structure of the body of the message.

Robustness of the Fusion Model: In dubious news items, the Master model tends to be more balanced, avoiding the extreme verdicts of the other two and offering greater generalisation capacity.

❌ The Bad (Weaknesses)

Vulnerability to Satire: The system "falls" into the trap of media outlets such as The Onion. By using grammatically perfect and serious language, the ISOT model classifies it as Real, demonstrating that AI confuses "professional form" with "truth."

Lack of "Common Sense": The models do not know that it is physically impossible for it to snow in space. This confirms that BERT is a linguistic expert, but has no knowledge of the real world or the laws of physics.

Stylistic Bias: The system remains sensitive to capital letters and emotional language (Taylor Swift case). If a fake news story is very well written, it is highly likely to fool the AI.
